<a href="https://colab.research.google.com/github/Vakhranev/Heritage/blob/main/%D0%98%D1%81%D0%BF%D1%80%D0%B0%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5%20%D0%BF%D0%B0%D1%80%D1%81%D0%B5%D1%80%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import json
import re
from collections import Counter

# Функция для приведения текста к нижнему регистру
def clean_text(text):
    return text.lower()

# Функция для токенизации текста с разделением знаков препинания
# Знаки препинания становятся отдельными токенами, но дефисы в словах сохраняются
def tokenize_text(text):
    return re.findall(r'\b\w+(?:-\w+)?\b|[.,:;!?"\(\)\[\]\{\}<>]', text)

# Функция для подсчёта биграмм в тексте
# Исключаем биграммы, содержащие знаки препинания
def count_bigrams(tokens):
    bigrams = []
    for i in range(len(tokens) - 1):
        if not re.match(r'[.,:;!?"\(\)\[\]\{\}<>]', tokens[i]) and not re.match(r'[.,:;!?"\(\)\[\]\{\}<>]', tokens[i + 1]):
            bigrams.append((tokens[i], tokens[i + 1]))
    bigram_freq = Counter(bigrams)
    return bigram_freq

# Загрузка значений t_score из файла
def load_t_scores(filename):
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            return json.load(f)
    except UnicodeDecodeError as e:
        print(f"UnicodeDecodeError: {e}")
        print("Попробуем открыть файл в другой кодировке...")
        with open(filename, 'r', encoding='latin1') as f:
            return json.load(f)

# Функция для проверки и корректировки биграммы с ё
def check_yo_in_bigram(bigram, t_scores):
    bigram_yo = bigram.replace('ё', 'е')  # Версия биграммы с "е"

    # Сначала проверяем исходную биграмму
    if bigram in t_scores:
        return t_scores[bigram]

    # Если исходной нет, проверяем версию с "е"
    if bigram_yo in t_scores:
        return t_scores[bigram_yo]

    # Если обе версии отсутствуют, возвращаем 0
    return 0

def process_america_texts(input_json, t_score_file, output_file):
    # Загрузка текстов
    with open(input_json, 'r', encoding='utf-8') as f:
        america_texts = json.load(f)

    # Загрузка t_scores
    t_scores = load_t_scores(t_score_file)

    result = {}

    for key, data in america_texts.items():
        # Исходный текст
        text = data["original_text"]

        # Приведение текста к нижнему регистру
        clean_text_content = clean_text(text)

        # Токенизация текста
        tokens = tokenize_text(clean_text_content)

        # Подсчёт частоты биграмм
        bigram_freq = count_bigrams(tokens)
        bigram_freq = {" ".join(bigram): freq for bigram, freq in bigram_freq.items()}

        # Формирование биграмм с t_score
        bigrams_unique = {}
        for bigram, freq in bigram_freq.items():
            t_score = check_yo_in_bigram(bigram, t_scores)  # Используем t_score с проверкой ё/е
            bigrams_unique[bigram] = {"t_score": t_score}

        # Сохранение результатов
        result[key] = {
            "text": text,
            "tokens": tokens,
            "bigrams_freq": bigram_freq,
            "bigrams_unique": bigrams_unique,
        }

    # Запись в файл
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=4)

# Пример использования
input_json = "lib_random_texts.json"
t_score_file = "t_scores.json"
output_file = "updated_lib_random_texts.json"

process_america_texts(input_json, t_score_file, output_file)

In [4]:
import json
import re
from collections import Counter

# Загрузка значений t_score из файла
def load_t_scores(filename):
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            return json.load(f)
    except UnicodeDecodeError as e:
        print(f"UnicodeDecodeError: {e}")
        print("Попробуем открыть файл в другой кодировке...")
        with open(filename, 'r', encoding='latin1') as f:
            return json.load(f)

# Функция для проверки и корректировки биграммы с ё
def check_yo_in_bigram(bigram, t_scores):
    bigram_yo = bigram.replace('ё', 'е')  # Версия биграммы с "е"

    # Сначала проверяем исходную биграмму
    if bigram in t_scores:
        return t_scores[bigram]

    # Если исходной нет, проверяем версию с "е"
    if bigram_yo in t_scores:
        return t_scores[bigram_yo]

    # Если обе версии отсутствуют, возвращаем 0
    return 0

def update_bigrams_unique(input_json, t_score_file, output_file):
    # Загрузка текстов
    with open(input_json, 'r', encoding='utf-8') as f:
        america_texts = json.load(f)

    # Загрузка t_scores
    t_scores = load_t_scores(t_score_file)

    for key, data in america_texts.items():
        if "bigrams_unique" in data:
            for bigram in data["bigrams_unique"]:
                t_score = check_yo_in_bigram(bigram, t_scores)  # Используем t_score с проверкой ё/е
                data["bigrams_unique"][bigram]["t_score"] = t_score

    # Запись в файл
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(america_texts, f, ensure_ascii=False, indent=4)

# Пример использования
input_json = "lib_random_texts.json"
t_score_file = "t_scores.json"
output_file = "updated_lib_random_texts.json"

update_bigrams_unique(input_json, t_score_file, output_file)

In [ ]:
import json

# Загрузка значений t_score из файла
def load_t_scores(filename):
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            return json.load(f)
    except UnicodeDecodeError as e:
        print(f"UnicodeDecodeError: {e}")
        print("Попробуем открыть файл в другой кодировке...")
        with open(filename, 'r', encoding='latin1') as f:
            return json.load(f)

# Функция для проверки биграммы с учётом ё/е
def check_yo_in_bigram(bigram, t_scores):
    bigram_yo = bigram.replace('ё', 'е')
    t_score = t_scores.get(bigram, None)

    if t_score is None and bigram_yo != bigram:
        return t_scores.get(bigram_yo, None)  # Проверяем версию с "е"

    return t_score

# Основной код
def check_bigram_t_score(t_score_file):
    # Загрузка t_scores
    t_scores = load_t_scores(t_score_file)

    while True:
        bigram = input("Введите биграмму (или 'exit' для выхода): ").strip()
        if bigram.lower() == 'exit':
            print("Выход из программы.")
            break

        t_score = check_yo_in_bigram(bigram, t_scores)
        if t_score is not None:
            print(f"t-score для биграммы '{bigram}': {t_score}")
        else:
            print(f"Биграмма '{bigram}' не найдена в файле.")

# Пример использования
t_score_file = "cleaned_t_score_file.json"
check_bigram_t_score(t_score_file)

Введите биграмму (или 'exit' для выхода): она еще
t-score для биграммы 'она еще': 0.09677664601101961
Введите биграмму (или 'exit' для выхода): exit
Выход из программы.
